<h1>Proximal Policy Optimization</h1>
    
In Lectures we have seen the concepts behind Reinforcement Learning and how the various algorthims work to maximise some objective. We've seen basic versions learn in gridworlds with look-up tables and more modern version learn control tasks with MLPs.<br>
In this lab we are taking things even further and introducing a very popular and widely used policy gradient algroithm, <a href="https://arxiv.org/pdf/1707.06347.pdf">Proximal Policy Optimization</a>. This algorthims is the culmulation of many improvements in policy gradient algorithms and contains many tricks to help quickly and stably train RL agents on a range of possible tasks. <br>
In this lab we'll talk through the different improvements that have been made over basic Actor Critic algorithms and then we'll look at what makes PPO special. 

In [ ]:
# Uncomment to install the procgen environments!
# !pip install procgen

In [ ]:
import math
import random

from procgen import ProcgenEnv
import gym
import imageio

import numpy as np
import time 
import torch
import torch.nn as nn

import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal, Categorical
from IPython.display import clear_output
import matplotlib.pyplot as plt

from collections import deque

In [ ]:
device = torch.device(0 if torch.cuda.is_available() else "cpu")

<img src="../data/PROCGEN.png" width="750" align="center">
<br>
In the lectures we introduced the <a href="https://openai.com/blog/procgen-benchmark/">Procgen</a> training environments, which were created to test the generalization ability of Reinforcement Learning.<br>
However, we never trained an RL agent to learn from these environments!


In [ ]:
# Procgen uses OpenAI gym and can create parallel instances of the same game, allowing us to collect many rollouts 
# at once, meaning we can train much faster and smoother!

# The name of the Procgen game
env_name = "coinrun"
# How many levels we wish to train on
num_levels = 20
# Hard or easy levels (see paper for the difference)
dist_mode = "easy"
# The number of parallel instances of the game we wish to create 
num_envs = 64
# Create the environment instance
envs = ProcgenEnv(num_envs=num_envs, env_name=env_name, start_level=0,
                  num_levels=num_levels, distribution_mode=dist_mode)

<h3>Network Architecture</h3>
We are taking the Large IMPALA Architecture (without LSTM) from the <a href="https://arxiv.org/pdf/1802.01561.pdf">Importance Weighted Actor-Learner Architectures</a> Paper which outlines a method to train RL agents in a Scalable and Distributed way.
Note that we are NOT using that method here, we are simply using the Network.<br>
This Network is Deep, but is still "light-weight" enough for RL, it also has very large residual skip connects.

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, 3, 1, 1)
        self.conv2 = nn.Conv2d(in_channels, in_channels, 3, 1, 1)
        self.relu = nn.ReLU()

    def forward(self, x_in):
        x = self.relu(x_in)
        x = F.relu(self.conv1(x))
        x_out = self.conv2(x)
        return x_in + x_out


class ConvBlock(nn.Module):
    def __init__(self, in_channels, block_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, block_channels, 3, 1, 1)
        self.max_pool = nn.MaxPool2d(3, 2, 1)
        self.res1 = ResBlock(block_channels)
        self.res2 = ResBlock(block_channels)

    def forward(self, x_in):
        x = self.conv1(x_in)
        x = self.max_pool(x)
        x = self.res1(x)
        return self.res2(x)


class ImpalaCnn64(nn.Module):
    def __init__(self, in_channels, num_outputs, base_channels=16):
        super(ImpalaCnn64, self).__init__()
        self.conv_block1 = ConvBlock(in_channels, base_channels)
        self.conv_block2 = ConvBlock(base_channels, 2*base_channels)
        self.conv_block3 = ConvBlock(2*base_channels, 2*base_channels)

        self.fc1 = nn.Linear(8*8*2*base_channels, 256)
        self.actor = nn.Linear(256, num_outputs)
        self.critic = nn.Linear(256, 1)
        self.SM = nn.Softmax(1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)

        x = x.reshape(x.size(0), -1)
        self.state = self.relu(self.fc1(x))

        value = self.critic(self.state)
        self.prob = self.SM(self.actor(self.state))
        dist = Categorical(probs=self.prob)
        return dist, value

<h3>Dataset Buffer</h3>
Just like we saw in Deep Q learning we will use a buffer to store all of the environment interactions. However, unlike DQNs all the data in the buffer will be collected by the same Policy. So we will completly refill the buffer after every training update.

In [ ]:
class ReplayBuffer:
    def __init__(self, data_names, buffer_size, mini_batch_size, device):
        self.data_keys = data_names
        self.data_dict = {}
        self.buffer_size = buffer_size
        self.mini_batch_size = mini_batch_size
        self.device = device

        self.reset()

    def reset(self):
        # Create a deque for each data type with set max length
        for name in self.data_keys:
            self.data_dict[name] = deque(maxlen=self.buffer_size)

    def buffer_full(self):
        return len(self.data_dict[self.data_keys[0]]) == self.buffer_size

    def data_log(self, data_name, data):
        # split tensor along batch into a list of individual datapoints
        data = data.cpu().split(1)
        # Extend the deque for data type, deque will handle popping old data to maintain buffer size
        self.data_dict[data_name].extend(data)

    def __iter__(self):
        batch_size = len(self.data_dict[self.data_keys[0]])
        batch_size = batch_size - batch_size % self.mini_batch_size

        ids = np.random.permutation(batch_size)
        ids = np.split(ids, batch_size // self.mini_batch_size)
        for i in range(len(ids)):
            batch_dict = {}
            for name in self.data_keys:
                c = [self.data_dict[name][j] for j in ids[i]]
                batch_dict[name] = torch.cat(c).to(self.device)
            batch_dict["batch_size"] = len(ids[i])
            yield batch_dict

    def __len__(self):
        return len(self.data_dict[self.data_keys[0]])

<h3> Helper Functions</h3>

In [ ]:
# Procgen returns a dictionary as the state, this fuction converts the rbg images [0, 255] into a tensor [0, 1]
def state_to_tensor(state_dict, device):
    if type(state_dict) is dict:
        state = torch.FloatTensor(state_dict['rgb'].transpose((0, 3, 1, 2))).to(device)
    else:
        state = torch.FloatTensor(state_dict.transpose((2, 0, 1))).unsqueeze(0).to(device)

    return state / 256  # Put tensor on gpu before divide - much faster


# Convert the Image Tensor to a uint8 image for saving
def tensor_to_unit8(tesnor):
    numpy_float = tesnor.squeeze(0).cpu().numpy().transpose((1, 2, 0))
    return (numpy_float * 255).astype(np.uint8)

# To test the agent we loop through all the training levels and an equivelant number of unseen levels
# Note this is not optimal as the training will wait untill this is done before continuing.
# With more training levels the time it takes to test will increase!
# Testing is usually done in a seperate process using the current saved checkpoint of the Policy parameters 
# (see IMPALA paper for a "full on" distributed method)
def run_tests(dist_mode, env_name, num_levels, train_test="train"):
    if train_test == "train":
        start_level = 0
    else:
        start_level = num_levels
    
    scores = []
    for i in range(num_levels):
        env = gym.make("procgen:procgen-" + env_name + "-v0", 
                       start_level=start_level + i, num_levels=1, distribution_mode=dist_mode)
        scores.append(test_agent(env))
        
    return np.mean(scores)

# Tests Policy once on the given environment
def test_agent(env, log_states=False):
    start_state = env.reset()
    state = state_to_tensor(start_state, device)
    
    if log_states:
        states_logger = [tensor_to_unit8(state)]
    
    done = False
    total_reward = 0
    with torch.no_grad():
        while not done:
            dist, _ = rl_model(state)  # Forward pass of actor-critic model
            action = dist.sample().item()

            next_state, reward, done, _ = env.step(action)
            total_reward += reward
            state = state_to_tensor(next_state, device)
            if log_states:
                states_logger.append(tensor_to_unit8(state))
                
    if log_states:
        return total_reward, states_logger
    else:
        return total_reward

<h3>PPO Clipped Surrogate Objective </h3>
PPO follows on from another algorithm called Trust Region Policy Optimization (TRPO) where they ask the question, <em>"how much should we trust this optimization step</em>?" RL is notoriously "noisy" and the wrong step could collapse our policy! PPO uses an approximation of the "Trust Region" step in TRPO (which can be difficult to compute) based on some heuristics (created using their experience of implementing RL algorithms) to perform a "safe" update step.<br>

The total combined optimization objective that we wish to <b>Maximise</b> ($L^{CLIP}$) is as follows:
<br>
<br>

\begin{equation*}
L^{CLIP}(\theta) = \hat{\mathbb{E}}_t[min \left(r_t(\theta)\hat{\mathbf{A}}_t, \:clip(r_t(\theta), 1-\epsilon, 1+\epsilon)\hat{\mathbf{A}}_t\right)] 
\end{equation*}

Where:
\begin{equation*}
r_t(\theta) = \frac{\pi_\theta(a_t, s_t)}{\pi_{\theta_{old}}(a_t, s_t)}
\end{equation*}
<br>
Also: $\hat{\mathbf{A}}_t$ is the Advantage (more on this soon) and $\epsilon$ is the clipping parameter.<br>

A visual representation of this objective is seen below.

<img src="../data/PPO_Clipping.png" width="750" align="center">

<br>
The best way to understand the objective is to see it implemented! Read the comments bellow to see how each part works!

In [ ]:
def ppo_loss(new_dist, actions, old_log_probs, advantages, clip_param):
    ########### Policy Gradient update for actor with clipping - PPO #############
    
    # 1. Find the new probability 
    # Work out the probability (log probability) that the agent will NOW take
    # the action it took during the rollout
    # We assume there has been some optimisation steps between when the action was taken and now so the
    # probability has probably changed
    new_log_probs = new_dist.log_prob(actions)
    
    # 2. Find the ratio of new to old - r_t(theta)
    # Calculate the ratio of new/old action probability (remember we have log probabilities here)
    # log(new_prob) - log(old_prob) = log(new_prob/old_prob)
    # exp(log(new_prob/old_prob)) = new_prob/old_prob
    # We use the ratio of new/old action probabilities (not just the log probability of the action like in
    # vanilla policy gradients) so that if there is a large difference between the probabilities then we can
    # take a larger/smaller update step
    # EG: If we want to decrease the probability of taking an action but the new action probability
    # is now higher than it was before we can take a larger update step to correct this
    ratio = (new_log_probs - old_log_probs).exp()

    # 3. Calculate the ratio * advantage - the first term in the MIN statement
    # We want to MAXIMISE the (Advantage * Ratio)
    # If the advantage is positive this corresponds to INCREASING the probability of taking that action
    # If the advantage is negative this corresponds to DECREASING the probability of taking that action
    surr1 = ratio * advantages

    # 4. Calculate the (clipped ratio) * advantage - the second term in the MIN statement
    # PPO goes a bit further, if we simply update update using the Advantage * Ratio we will sometimes
    # get very large or very small policy updates when we don't want them
    #
    # EG1: If we want to increase the probability of taking an action but the new action probability
    # is now higher than it was before we will take a larger step, however if the action probability is
    # already higher we don't need to keep increasing it (large output values can create instabilities).
    #
    # EG2: You can also consider the opposite case where we want to decrease the action probability
    # but the probability has already decreased, in this case we will take a smaller step than before,
    # which is also not desirable as it will slow down the "removal" (decreasing the probability)
    # of "bad" actions from our policy.
    surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * advantages
    
    # 5. Take the minimum of the two "surrogate" losses
    # PPO therefore clips the upper bound of the ratio when the advantage is positive
    # and clips the lower bound of the ratio when the advantage is negative so our steps are not too large
    # or too small when necessary, it does this by using a neat trick of simply taking the MIN of two "surrogate"
    # losses which chooses which loss to use!
    actor_loss = torch.min(surr1, surr2)
    
    # 6. Return the Expectation over the batch
    return actor_loss.mean()

In [ ]:
def clipped_critic_loss(new_value, old_value, returns, clip_param):
    ########### Value Function update for critic with clipping #############
    
    # To help stabalise the training of the value function we can do a similar thing as the clipped objective
    # for PPO - Note: this is NOT nessisary but does help!
        
    # 1. MSE/L2 loss on the current value and the returns
    vf_loss1 = (new_value - returns).pow(2.)
    
    # 2. MSE/L2 loss on the clipped value and the returns
    # Here we create an "approximation" of the new value (aka the current value) by finding the difference
    # between the "new" and "old" value and adding a clipped amount back to the old value
    vpredclipped = old_value + torch.clamp(new_value - old_value, -clip_param, clip_param)
    # Note that we ONLY backprop through the new value
    vf_loss2 = (vpredclipped - returns).pow(2.)
    
    # 3. Take the MAX between the two losses
    # This trick has the effect of only updating the current value DIRECTLY if is it WORSE (higher error)
    # than the old value. 
    # If the old value was worse then the "approximation" will be worse and we update
    # the new value only a little bit!
    critic_loss = torch.max(vf_loss1, vf_loss2)
    
    # 4. Return the Expectation over the batch
    return critic_loss.mean()

<h3>Generalised Advantage Estimate</h3>
Another Improvement that implementation of PPO often use is the Generalised Advantage Estimate (GAE) from the paper
<a href="https://arxiv.org/pdf/1506.02438.pdf">High Dimensional Continuous Control Using Generalized Advantage Estimation</a> (This is also a great paper for understanding policy optimisation algorithms and RL in general).<br>
GAE creates an estimate of the Advantage that is less noisy be combining Advantage you get by using the returns approximation from TD learning and "real" returns update.<br>

\begin{equation*}
\hat{\mathbf{A}}_t^{GAE(\gamma, \tau)} = \sum_{l=0}^{\infty}{(\gamma\tau)^l\delta^V_{t+1}}
\end{equation*}
<br>
Where $\delta^V_{t+1}$ is our TD update (and also the an approximation of the Advantage):
<br>

\begin{equation*}
\delta^V_{t+1} =  r_t + \gamma\mathbf{V}(s_{t+1})-\mathbf{V}(s_t)
\end{equation*}
<br>
\begin{equation*}
\mathbf{R}_t \: \tilde{=}  \: r_t + \gamma\mathbf{V}(s_{t+1})
\end{equation*}
<br>
And $\gamma$ is the decay rate of future reward, $0 < \gamma < 1$,  and $\tau$ is the decay rate of the sum of $\delta^V_{t+1}$, $0 < \tau < 1$.<br>

To get a better idea of what $\tau$ does lets look at two cases outlined in the paper, when $\tau = 0$ and $\tau = 1$:

\begin{equation*}
\hat{\mathbf{A}}_t^{GAE(\gamma, 0)} = r_t + \gamma\mathbf{V}(s_{t+1})-\mathbf{V}(s_t)
\end{equation*}
<br>
\begin{equation*}
\hat{\mathbf{A}}_t^{GAE(\gamma, 1)} = \sum_{l=0}^{\infty}{\gamma^lr_{t+l}}-\mathbf{V}(s_t)
\end{equation*}

You can see that $\tau$ controls the mixing between a basic advantage calculation (which can be noisy) and an advantage calculation using the Value and rewards as an appromimation of the returns (which is biased). <br>
To seperate out different levels within a single environment so that we don't calculate the GAE over the WHOLE sequence we use "masks" (1 or 0) to "zero" the GAE and start again. More on the training environments later!

In [ ]:
def compute_gae(next_value, rewards, masks, values, gamma=0.999, tau=0.95):
    # Similar to calculating the returns we can start at the end of the sequence and go backwards
    gae = 0
    returns = deque()
    gae_logger = deque()

    for step in reversed(range(len(rewards))):
        # Calculate the current delta value
        delta = rewards[step] + gamma * next_value * masks[step] - values[step]
        
        # The GAE is the decaying sum of these delta values
        gae = delta + gamma * tau * masks[step] * gae
        
        # Get the new next value
        next_value = values[step]
        
        # If we add the value back to the GAE we get a TD approximation for the returns
        # which we can use to train the Value function
        returns.appendleft(gae + values[step])
        gae_logger.appendleft(gae)

    return returns, gae_logger

<h3>Update Loop</h3>
Using the loss functions we created before we will now train our actor/critic

In [ ]:
def ppo_update(data_buffer, ppo_epochs, clip_param):
    for _ in range(ppo_epochs):
        for data_batch in data_buffer:
            # Forward pass of input state observationsequence
            new_dist, new_value = rl_model(data_batch["states"])

            # Most Policy gradient algorithms include a small "Entropy bonus" to increases the "entropy" of 
            # the action distribution, aka the "randomness"
            # This ensures that the actor does not converge to taking the same action everytime and
            # maintains some ability for "exploration" of the policy
            
            # Determine expectation over the batch of the action distribution entropy
            entropy = new_dist.entropy().mean()

            actor_loss = ppo_loss(new_dist, data_batch["actions"], data_batch["log_probs"], data_batch["advantages"],
                                  clip_param)

            critic_loss = clipped_critic_loss(new_value, data_batch["values"], data_batch["returns"], clip_param)

            # These techniques allow us to do multiple epochs of our data without huge update steps throwing off our
            # policy/value function (gradient explosion etc).
            # It can also help prevent "over-fitting" to a single batch of observations etc, 
            # RL boot-straps itself and the noisy "ground truth" targets (if you can call them that) will
            # shift overtime and we need to make sure our actor-critic can quickly adapt, over-fitting to a
            # single batch of observations will prevent that
            agent_loss = critic_loss - actor_loss - 0.01 * entropy

            optimizer.zero_grad()
            agent_loss.backward()
            # Clip gradient norm to further prevent large updates
            nn.utils.clip_grad_norm_(rl_model.parameters(), 40)
            optimizer.step()

<h3>Hyperparameters</h3>

In [ ]:
# Hyperparameters, PPO has many many hyperparameters
lr = 5e-4
# How many minibatchs (therefore optimization steps) we want per epoch 
num_mini_batch = 8
# Total number of steps during the rollout phase 
num_steps = 256 
# Number of Epochs for training
ppo_epochs = 3

# PPO parameters
gamma = 0.999
tau = 0.95
clip_param = 0.2

# Training parameters
max_frames = 10e6
frames_seen = 0
rollouts = 0

# Score loggers
test_score_logger = []
train_score_logger = []
frames_logger = []

# Set the size of the FIFO databuffer to the total number of steps for a single batch of rollouts
# so that every episode the buffer has completely reset
buffer_size = num_steps * num_envs
# Calculate the size of each minibatch  - usually very big - 2048!
mini_batch_size = buffer_size // num_mini_batch

# Define the data we wish to collect for the databuffer
data_names = ["states", "actions", "log_probs", "values", "returns", "advantages"]
data_buffer = ReplayBuffer(data_names, buffer_size, mini_batch_size, device)

# Create the actor critic Model and optimizer
rl_model = ImpalaCnn64(3, 15).to(device)
optimizer = optim.Adam(rl_model.parameters(), lr=lr)

<h3>The Main Training Loop</h3>
To understand the structure of the Rollout/Training Cycle it is important to know how the Procgen environments work and how the rollout/training structure works.
<br>
We have created a number of environments that will run in seperate processes on your computer. With every environment step you will get a dictionary that defines the current state. Within the dictionary is a batch of images that are the current game frames for all environments. The Procgen environments will run continuously, that is, for every environment as soon as one game/level is completed a new one starts. Therefore we do not use a "done" signal to determine when to stop as the level in one environment may be compleated before another.
<br><br>
By instead stopping after a number of timesteps we have another problem, we may have cut a rollout in half, stopping before it is over! Because of this we won't know if we could have gotten a Reward and therefore don't know what the true Returns would be. However, we are using a TD approximate of the Returns, that is we are using the Value function as an approximation of the Returns and so do not need to complete the level every time in order to train, the algorithm just uses it's best guess. As long as, on average, we fully complete the level within the rollout we can still train. Most of the levels in the different Procgen games can be completed within 256 steps and so this is the max  number of steps we use! 


In [ ]:
start_time = time.time()

while frames_seen < max_frames:
    rl_model.train()
    # Initialise state
    start_state = envs.reset()
    state = state_to_tensor(start_state, device)

    # Create data loggers - deques a bit faster than lists
    log_probs = deque()
    values = deque()
    states = deque()
    actions = deque()
    rewards = deque()
    masks = deque()
    step = 0
    done = np.zeros(num_envs)

    print("Rollout!")
    with torch.no_grad():  # Don't need computational graph for roll-outs
        while step < num_steps:
            #  Masks so we can separate out multiple games in the same environment
            dist, value = rl_model(state)  # Forward pass of actor-critic model
            action = dist.sample()  # Sample action from distribution

            # Take the next step in the env
            next_state, reward, done, env_info = envs.step(action.cpu().numpy())

            # Work out reward, we are just setting the reward to be either
            # -1, 1 or 0 for negative, positive or no reward
            reward = torch.FloatTensor(((reward > 0).astype("float64") - (reward < 0).astype("float64")))

            # Log data
            log_prob = dist.log_prob(action)
            log_probs.append(log_prob)
            states.append(state)
            actions.append(action)
            values.append(value)
            rewards.append(reward.unsqueeze(1).to(device))
            current_mask = torch.FloatTensor(1 - done).unsqueeze(1).to(device)
            masks.append(current_mask)

            state = state_to_tensor(next_state, device)
            step += 1

        # Get value at time step T+1
        _, next_value = rl_model(state)
        # Calculate the returns/gae
        returns, advantage = compute_gae(next_value, rewards, masks, values, gamma=gamma, tau=tau)

        data_buffer.data_log("states", torch.cat(list(states)))
        data_buffer.data_log("actions", torch.cat(list(actions)))
        data_buffer.data_log("returns", torch.cat(list(returns)))
        data_buffer.data_log("log_probs", torch.cat(list(log_probs)))
        data_buffer.data_log("values", torch.cat(list(values)))

        advantage = torch.cat(list(advantage)).squeeze(1)
        # Normalising the Advantage helps stabalise training!
        data_buffer.data_log("advantages", (advantage - advantage.mean()) / (advantage.std() + 1e-8))
        
        # Update the frames counter
        # We normaly base how long to train for by counting the number of "environment interactions"
        # In our case we can simply counte how many game frames we have received from the environment
        frames_seen += advantage.shape[0]
    
    # We train after every batch of rollouts
    # With the stabalisation techniques in PPO we can "safely" take many steps with a single
    # batch of rollouts, therefore we usualy train with the data over multiple epochs whereas basic
    # actor critic methods only use one epoch.
    print("Training")
    ppo_update(data_buffer, ppo_epochs, clip_param)
    rollouts += 1

    if rollouts % 10 == 0:
        print("Testing")
        rl_model.eval()
        test_score = run_tests(dist_mode, env_name, num_levels, train_test="test")
        train_score = run_tests(dist_mode, env_name, num_levels, train_test="train")

        test_score_logger.append(test_score)
        train_score_logger.append(train_score)
        frames_logger.append(frames_seen)
        print("Trained on %d Frames, Test/Train Scores [%d/%d]" 
            %(frames_seen, test_score, train_score))

        time_to_end = ((time.time() - start_time) / frames_seen) * (max_frames - frames_seen)
        print("Time to end: %dh:%dm" % (time_to_end // 3600, (time_to_end % 3600) / 60))

In [ ]:
plt.plot(frames_logger, test_score_logger)
plt.plot(frames_logger, train_score_logger)
plt.legend(["Testing", "Training"])

In [ ]:
env = gym.make("procgen:procgen-" + env_name + "-v0", start_level=200, num_levels=1, distribution_mode=dist_mode)
score, states = test_agent(env, True)
imageio.mimsave(env_name + "_" + dist_mode + "_rollout.gif", states, fps=15)